<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 35px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
      Sequence Labelling - MLM
  </div> 
  
<div style="
      font-weight: normal; 
      font-size: 25px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
      Training on Clinical Trials (Albert-small)
  </div> 


  <div style="
      font-size: 15px; 
      line-height: 12px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
  Jean-baptiste AUJOGUE - Hybrid Intelligence
  </div> 

  
  <div style=" float:right; 
      font-size: 12px; 
      line-height: 12px; 
  padding: 10px 15px 8px;">
  December 2022
  </div>

<a id="TOC"></a>

#### Table Of Content

1. [Dataset](#data) <br>
2. [ALBERT finetuning](#albert) <br>
3. [Inference](#inference) <br>



#### Reference

- Hugginface full list of [tutorial notebooks](https://github.com/huggingface/transformers/tree/main/notebooks) (see also [here](https://huggingface.co/docs/transformers/main/notebooks#pytorch-examples))
- Huggingface full list of [training scripts](https://github.com/huggingface/transformers/tree/main/examples/pytorch)
- Huggingface [tutorial notebook](https://github.com/huggingface/notebooks/blob/main/examples/language_modeling_from_scratch.ipynb) on language models
- Huggingface [course](https://huggingface.co/course/chapter7/3?fw=tf) on language models
- Huggingface [training script](https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py) on language models
- Albert [original training protocol](https://github.com/google-research/albert)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import re
import random
import copy
import string
from itertools import chain

# data
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import (
    Dataset, 
    DatasetDict,
    ClassLabel, 
    Features, 
    Sequence, 
    Value,
    load_from_disk,
)
from transformers import AlbertConfig, AutoConfig, DataCollatorForLanguageModeling

# DL
import torch
import transformers
from transformers import (
    AutoTokenizer, 
    AutoModelForMaskedLM, 
    TrainingArguments, 
    Trainer,
    pipeline,
    set_seed,
)
import evaluate

# viz
from IPython.display import HTML

C:\Users\jb\miniconda3\envs\transformers_nlp\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Transformers settings

In [3]:
transformers.__version__

'4.22.2'

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
# make training deterministic
set_seed(42)

#### Custom paths & imports

In [10]:
path_to_repo = os.path.dirname(os.path.dirname(os.getcwd()))
path_to_data = os.path.join(path_to_repo, 'datasets')
path_to_save = os.path.join(path_to_repo, 'saves', 'MLM')
path_to_src  = os.path.join(path_to_repo, 'src')

In [11]:
sys.path.insert(0, path_to_src)

#### Constants

In [12]:
dataset_name = 'clinical-trials-ctti'
base_model_name = "albert-base-v2"
final_model_name = "clinical-trials-albert-small"

<a id="data"></a>

# 1. Dataset

[Table of content](#TOC)

We generate a collection of instances of the `datasets.Dataset` class. 

Note that these are different from the fairly generic `torch.utils.data.Dataset` class. 

## 1.1 Load Clinical Trials corpus

[Table of content](#TOC)

In [31]:
df_trials = pd.read_csv(os.path.join(path_to_data, '{}.tsv'.format(dataset_name)), sep = "\t")
df_trials = df_trials.fillna('')


df_trials.head(10)

,Id,Summary,Description,IE_criteria,Condition,Purpose,Intervention
0,NCT0000xxxx/NCT00000102.xml,This study will test the ability of extended r...,This protocol is designed to assess both acute...,diagnosed with Congenital Adrenal Hyperplasia ...,Congenital Adrenal Hyperplasia,Treatment,Nifedipine
1,NCT0000xxxx/NCT00000104.xml,Inner city children are at an increased risk f...,,,Lead Poisoning,,ERP measures of attention and memory
2,NCT0000xxxx/NCT00000105.xml,The purpose of this study is to learn how the ...,Patients will receive each vaccine once only c...,Patients must have a diagnosis of cancer of an...,Cancer,,Intracel KLH Vaccine
3,NCT0000xxxx/NCT00000106.xml,Recently a non-toxic system for whole body hyp...,,,Rheumatic Diseases,Treatment,Whole body hyperthermia unit
4,NCT0000xxxx/NCT00000107.xml,Adults with cyanotic congenital heart disease ...,,Resting blood pressure below 140/90,"Heart Defects, Congenital",,
5,NCT0000xxxx/NCT00000108.xml,The purpose of this research is to find out wh...,,Postmenopausal and preferably on hormone repla...,Cardiovascular Diseases,Prevention,Exercise
6,NCT0000xxxx/NCT00000110.xml,The purpose of this pilot investigation is to ...,,Healthy volunteers (developmental phase)\nHeal...,Obesity,Treatment,magnetic resonance spectroscopy
7,NCT0000xxxx/NCT00000111.xml,The purpose of this study is to see if we can ...,,Lack sufficient attached keratinized tissue at...,Mouth Diseases,Treatment,Oral mucosal graft
8,NCT0000xxxx/NCT00000112.xml,The prevalence of obesity in children is reach...,,Obesity: BM +/- 95% for age general good health,Obesity,,
9,NCT0000xxxx/NCT00000113.xml,To evaluate whether progressive addition lense...,Myopia (nearsightedness) is an important publi...,,Myopia,Treatment,Progressive Addition Lenses


In [32]:
# df_trials = df_trials.iloc[:1000,:]

In [33]:
df_trials.shape

(430108, 7)

In [34]:
texts = df_trials[['Summary', 'Description', 'IE_criteria']].values.tolist()
texts = [t.strip() for ts in texts for t in ts if len(t.strip())>=50]
# texts = df_trials.IE_criteria.tolist()
# texts = [t for t in texts if len(t.strip())>=50]
len(texts)

1041969

In [35]:
dataset = Dataset.from_dict(
    {'text': texts}, 
    features = Features({'text': Value(dtype = 'string')}),
)

In [36]:
dataset[:3]

{'text': ['This study will test the ability of extended release nifedipine (Procardia XL), a blood pressure medication, to permit a decrease in the dose of glucocorticoid medication children take to treat congenital adrenal hyperplasia (CAH).',
  'This protocol is designed to assess both acute and chronic effects of the calcium channel antagonist, nifedipine, on the hypothalamic-pituitary-adrenal axis in patients with congenital adrenal hyperplasia. The multicenter trial is composed of two phases and will involve a double-blind, placebo-controlled parallel design. The goal of Phase I is to examine the ability of nifedipine vs. placebo to decrease adrenocorticotropic hormone (ACTH) levels, as well as to begin to assess the dose-dependency of nifedipine effects. The goal of Phase II is to evaluate the long-term effects of nifedipine; that is, can attenuation of ACTH release by nifedipine permit a decrease in the dosage of glucocorticoid needed to suppress the HPA axis? Such a decrease wo

## 1.2 Build Clinical-Albert-small tokenizer

[Table of content](#TOC)


In [9]:
def batch_iterator(dataset, batch_size = 512):
    for i in range(0, len(dataset), batch_size):
        yield dataset[i: i + batch_size]['text']

In [10]:
# tokenizer = AutoTokenizer.from_pretrained(base_model_name)
# tokenizer = tokenizer.train_new_from_iterator(batch_iterator(dataset, batch_size = 512), vocab_size = 5000)
# tokenizer.save_pretrained(os.path.join(path_to_save, final_model_name, 'tokenizer'))

In [15]:
# texts[0], tokenizer.decode(tokenizer(dataset[0]["text"])['input_ids'])

In [13]:
tokenizer = AutoTokenizer.from_pretrained(os.path.join(path_to_save, final_model_name, 'tokenizer'))

## 1.3 Tokenize corpus

[Table of content](#TOC)


In [16]:
# We use this option because DataCollatorForLanguageModeling (see below) is more efficient 
# when it receives the `special_tokens_mask`.
def tokenize_text(examples, tokenizer):
    # Remove empty lines
    examples['text'] = [
        t for t in examples['text'] if len(t) > 0 and not t.isspace()
    ]
    return tokenizer(examples["text"], return_special_tokens_mask = True)

In [17]:
# tokenized_dataset = dataset.map(lambda examples: tokenize_text(examples, tokenizer), batched = True, remove_columns = ["text"])

By contrast to the generic BIO annotated data, this new data depends on the tokenizer, and is therefore _model-specific_.

_Note_: the argument `remove_columns = ["text"]` is mandatory, in order to have each item of the dataset have same length.

## 1.4 Form blocks of constant length

[Table of content](#TOC)


In [18]:
def group_texts(examples, block_size):
    # Concatenate all texts.
    keys = [k for k in examples.keys() if k != 'text']
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[keys[0]])
    
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    
    # Split by chunks of max_len.
    result = {
        k: [t[i : i+block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [19]:
# mlm_dataset = tokenized_dataset.map(lambda examples: group_texts(examples, block_size = 512), batched = True)
# mlm_dataset.save_to_disk(os.path.join(path_to_data, 'tmp', dataset_name))

In [14]:
mlm_dataset = load_from_disk(os.path.join(path_to_data, 'tmp', dataset_name))

In [15]:
# texts[0], texts[1]

In [16]:
print(tokenizer.decode(mlm_dataset[0]["input_ids"]), tokenizer.decode(mlm_dataset[0]["labels"]))

[CLS] this study will test the ability of extended release nifedipine (procardia xl), a blood pressure medication, to permit a decrease in the dose of glucocorticoid medication children take to treat congenital adrenal hyperplasia (cah).[SEP][CLS] this protocol is designed to assess both acute and chronic effects of the calcium channel antagonist, nifedipine, on the hypothalamic-pituitary-adrenal axis in patients with congenital adrenal hyperplasia. the multicenter trial is composed of two phases and will involve a double-blind, placebo-controlled parallel design. the goal of phase i is to examine the ability of nifedipine vs. placebo to decrease adrenocorticotropic hormone (acth) levels, as well as to begin to assess the dose-dependency of nifedipine effects. the goal of phase ii is to evaluate the long-term effects of nifedipine; that is, can attenuation of acth release by nifedipine permit a decrease in the dosage of glucocorticoid needed to suppress the hpa axis? such a decrease wo

In [17]:
print(mlm_dataset[0])

{'input_ids': [2, 34, 30, 16, 129, 6, 621, 7, 2773, 1772, 5, 31, 17, 37, 39, 17, 1903, 202, 2829, 811, 17, 9, 5, 82, 42, 110, 5, 9, 85, 278, 358, 10, 13, 201, 879, 5, 9, 725, 14, 6, 118, 7, 4009, 358, 200, 515, 13, 1085, 2315, 3955, 587, 2153, 17, 9, 227, 9, 48, 67, 3, 2, 34, 281, 24, 586, 13, 175, 194, 255, 12, 186, 156, 7, 6, 2122, 3839, 3097, 10, 5, 31, 17, 37, 39, 17, 1903, 10, 40, 6, 983, 155, 53, 9, 1636, 22, 36, 1288, 17, 15, 438, 22, 9, 23, 114, 1628, 3673, 14, 29, 19, 2315, 3955, 587, 2153, 17, 9, 11, 6, 1927, 121, 24, 944, 2668, 7, 117, 154, 8, 12, 16, 2672, 5, 9, 1760, 228, 1121, 1931, 467, 11, 6, 631, 7, 154, 5, 17, 24, 13, 675, 6, 621, 7, 5, 31, 17, 37, 39, 17, 1903, 1935, 228, 13, 725, 5, 9, 23, 114, 31, 20, 3050, 75, 20, 3350, 1179, 277, 32, 155, 28, 276, 10, 38, 206, 38, 13, 2434, 13, 175, 6, 118, 22, 2194, 1194, 7, 5, 31, 17, 37, 39, 17, 1903, 156, 11, 6, 631, 7, 154, 5, 17, 17, 24, 13, 127, 6, 754, 156, 7, 5, 31, 17, 37, 39, 17, 1903, 71, 46, 24, 10, 100, 41, 752, 61,

In [18]:
len(mlm_dataset)

421077

<a id="albert"></a>

# 2. ALBERT-small training

[Table of content](#TOC)

#### Tested combinations

- 1.4M parameter model: converges fast (1 epoch) towards confusion score~=2.2. Issue : Finetuning of NER on Chia hard, stuck to high training error and/or provides evaluation errors
- 3.5M parameter model: gets stuck at confusion score~=5.9. Training args : block_size = 512, bs = 16, lr = 1e-4, grad_acc_step = 4, warmup_step = 500, num_layer = 8

## 2.1 Build Clinical-Albert-small model

[Table of content](#TOC)

In [19]:
# original Albert config
config = AutoConfig.from_pretrained(base_model_name)
config

AlbertConfig {
  "_name_or_path": "albert-base-v2",
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu_new",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 0,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.22.2",
  "type_vocab_size": 2,
  "vocab_size": 30000
}

In [20]:
# # a smaller Albert config
config = AlbertConfig(
    attention_probs_dropout_prob = 0.05, # 0
    pad_token_id = 0,
    bos_token_id = 2,
    eos_token_id = 3,
    classifier_dropout_prob = 0.1,
    down_scale_factor = 1,
    embedding_size = 128,
    gap_size = 0,
    hidden_act = 'gelu_new',
    hidden_dropout_prob = 0.05,
    hidden_size = 384, # 768,
    initializer_range = 0.02,
    inner_group_num = 1,
    intermediate_size = 1536, # 3072,
    layer_norm_eps = 1e-12,
    max_position_embeddings = 512,
    model_type = 'albert',
    net_structure_type = 0,
    num_attention_heads = 8, # 12
    num_hidden_groups = 1,
    num_hidden_layers = 8, # 12
    num_memory_blocks = 0,
    position_embedding_type = 'absolute',
    transformers_version = '4.22.2',
    type_vocab_size = 2,
    vocab_size = 5000, # 30000,
)
model = AutoModelForMaskedLM.from_config(config)

In [21]:
model.num_parameters()

2584584

In [22]:
model = model.to(device)

In [23]:
model
# model.albert.embeddings

AlbertForMaskedLM(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(5000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.05, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=384, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=384, out_features=384, bias=True)
                (key): Linear(in_features=384, out_features=384, bias=True)
                (value): Linear(in_features=384, out_features=384, bias=True)


## 2.2. Model training

[Table of content](#TOC)

`Albert-vase-v2` training parameters as provided in https://github.com/google-research/albert/blob/master/run_pretraining.py : 
- max_predictions_per_seq = `20`
- train_batch_size = `4096`
- optimizer = `"lamb"`
- learning_rate = `0.00176`
- poly_power = `1.0`
- num_train_steps = `125000`
- num_warmup_steps = `3125`
- start_warmup_step = `0`
- iterations_per_loop = `1000`

The original optimizer is `lamb`, which was designed for very large batch size, see the [Lamb paper](https://arxiv.org/pdf/1904.00962.pdf), but we use here the default [AdamW](https://huggingface.co/docs/transformers/main_classes/optimizer_schedules#transformers.AdamW) optimizer with [linear learning rate decay](https://huggingface.co/docs/transformers/v4.23.1/en/main_classes/optimizer_schedules#transformers.get_linear_schedule_with_warmup), as specified in the [Trainer class documentation](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Trainer.optimizers). See the [AdamW paper](https://arxiv.org/pdf/1711.05101.pdf).

In [24]:
batch_size = 16

In [33]:
model = model.train()

In [25]:
args = TrainingArguments(
    os.path.join(path_to_save, f"{final_model_name}-finetuned"),
    evaluation_strategy = "no",
    learning_rate = 5e-5,
    num_train_epochs = 1,
    warmup_steps = 1500,
    gradient_accumulation_steps = 1,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    save_strategy = 'no',
    logging_steps = 100,
    seed = 42,
    data_seed = 23,
)

In [26]:
trainer = Trainer(
    model,
    args,
    data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm_probability = 0.15),
    train_dataset = mlm_dataset,
)

Some remarks:

- The `data_collator` is the object used to batch elements of the training & evaluation datasets.
- The `tokenizer` is provided in order to automatically pad the inputs to the maximum length when batching inputs, and to have it saved along the model, which makes it easier to rerun an interrupted training or reuse the fine-tuned model.

In [27]:
torch.cuda.empty_cache()

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `AlbertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `AlbertForMaskedLM.forward`,  you can safely ignore this message.
C:\Users\jb\miniconda3\envs\transformers_nlp\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 421077
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 26318
You're using a AlbertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encod

Step,Training Loss
100,8.487700
200,8.323300
300,8.207800
400,8.072000
500,7.910000
600,7.719100
700,7.503900
800,7.279200
900,7.071800
1000,6.869600


In [64]:
model = model.to('cpu')

In [65]:
model.save_pretrained(os.path.join(path_to_save, final_model_name, 'model'))

Configuration saved in C:\Users\jb\Desktop\NLP\Internal - Transformers for NLP\saves\MLM\clinical-trials-albert-small\model\config.json
Model weights saved in C:\Users\jb\Desktop\NLP\Internal - Transformers for NLP\saves\MLM\clinical-trials-albert-small\model\pytorch_model.bin


In [82]:
model = model.to(device)

In [85]:
args = TrainingArguments(
    os.path.join(path_to_save, f"{final_model_name}-finetuned"),
    evaluation_strategy = "no",
    learning_rate = 5e-5,
    num_train_epochs = 1,
    warmup_steps = 0,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    save_strategy = 'no',
    logging_steps = 100,
    seed = 421,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [86]:
trainer = Trainer(
    model,
    args,
    data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm_probability = 0.15),
    train_dataset = mlm_dataset,
)

In [87]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `AlbertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `AlbertForMaskedLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 1695419
  Num Epochs = 1
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 26491


Step,Training Loss
100,4.892800
200,4.798500
300,4.810300
400,4.760200
500,4.775800
600,4.781700
700,4.768100
800,4.733300
900,4.738200
1000,4.721500




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=26491, training_loss=4.0612693332445895, metrics={'train_runtime': 8817.9899, 'train_samples_per_second': 192.268, 'train_steps_per_second': 3.004, 'total_flos': 3600006155175936.0, 'train_loss': 4.0612693332445895, 'epoch': 1.0})

<a id="inference"></a>

# 3. Inference

[Table of content](#TOC)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(os.path.join(path_to_save, final_model_name, 'tokenizer'))
model = AutoModelForMaskedLM.from_pretrained(os.path.join(path_to_save, final_model_name, 'model'))

In [66]:
mlm = pipeline(
    task = 'fill-mask', 
    model = model, 
    tokenizer = tokenizer,
    framework = 'pt',
)

In [67]:
sent = 'Polyneuropathy of other causes, including but not limited to hereditary demyelinating neuropathies, neuropathies secondary to infection or systemic disease, diabetic neuropathy, drug- or toxin-induced neuropathies, multifocal motor neuropathy, monoclonal gammopathy of uncertain significance, lumbosacral radiculoplexus neuropathy, pure sensory CIDP and acquired demyelinating symmetric (DADS) neuropathy (also known as distal CIDP).'
sent = f'Polyneuropathy of other causes, including but not limited to {mlm.tokenizer.mask_token} demyelinating neuropathies,  {mlm.tokenizer.mask_token} secondary to infection or systemic {mlm.tokenizer.mask_token}, diabetic neuropathy, drug- or toxin-induced neuropathies, multifocal motor {mlm.tokenizer.mask_token}, monoclonal gammopathy of uncertain significance, lumbosacral radiculoplexus neuropathy, pure sensory CIDP and acquired demyelinating symmetric (DADS) neuropathy (also known as distal CIDP).'
mlm(sent, top_k = 5)

[[{'score': 0.05104012042284012,
   'token': 5,
   'token_str': '',
   'sequence': '[CLS] polyneuropathy of other causes, including but not limited to  demyelinating neuropathies,[MASK] secondary to infection or systemic[MASK], diabetic neuropathy, drug- or toxin-induced neuropathies, multifocal motor[MASK], monoclonal gammopathy of uncertain significance, lumbosacral radiculoplexus neuropathy, pure sensory cidp and acquired demyelinating symmetric (dads) neuropathy (also known as distal cidp).[SEP]'},
  {'score': 0.04530152678489685,
   'token': 20,
   'token_str': 'o',
   'sequence': '[CLS] polyneuropathy of other causes, including but not limited too demyelinating neuropathies,[MASK] secondary to infection or systemic[MASK], diabetic neuropathy, drug- or toxin-induced neuropathies, multifocal motor[MASK], monoclonal gammopathy of uncertain significance, lumbosacral radiculoplexus neuropathy, pure sensory cidp and acquired demyelinating symmetric (dads) neuropathy (also known as dist

[Table of content](#TOC)